# Voice input and output feature

In [1]:
import importlib
import utils
importlib.reload(utils)
from utils import get_agent

In [2]:
from utils import init_qa_chain, get_agent

qa_chain = init_qa_chain()
agent = get_agent(qa_chain)


/Users/test/Desktop/ironhack_labs/YouTube_ChatBot_Final/notebooks/utils.py:21: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=keys["OPENAI_API_KEY"])
/Users/test/Desktop/ironhack_labs/YouTube_ChatBot_Final/notebooks/utils.py:27: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model="gpt-4", openai_api_key=key

In [ ]:
#voice input
# !pip install openai-whisper pyaudio pyttsx3
# !pip install sounddevice wavio

### Creates user audio recording functionality

In [3]:
import whisper
import sounddevice as sd
import numpy as np
import wavio
from gtts import gTTS
from playsound import playsound
import os
import uuid
import openai


import openai
import os

openai.api_key = os.getenv("OPENAI_API_KEY")


# Load Whisper model
model = whisper.load_model("base")

def record_audio(duration=5, fs=16000):
    print("🎙️ Recording...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1)
    sd.wait()
    wavio.write("/Users/test/Desktop/ironhack_labs/YouTube_ChatBot_Final/audio/user_input.wav", recording, fs, sampwidth=2)
    print("🎙️ Recording finished")

def transcribe_audio(filepath="/Users/test/Desktop/ironhack_labs/YouTube_ChatBot_Final/audio/user_input.wav"):
    print("📝 Transcribing...")
    result = model.transcribe(filepath)
    text = result["text"].strip()
    print(f"🗣️ You said: {text}")
    return text

def speak_text(text):
    filename = f"{uuid.uuid4().hex}.mp3"
    tts = gTTS(text=text, lang='en')
    tts.save(filename)
    playsound(filename)
    os.remove(filename)

def shorten_text(text, max_ratio=0.5):
    # Prompt the agent or LLM to summarize or shorten the text
    prompt = f"Please shorten the following text to about {int(max_ratio*100)}% of its length, keeping the key information:\n\n{text}"
    shortened = agent.run(prompt)
    return shortened

def answer_with_sources(question):
    # full_answer = agent.run(question)
    prompt = f"Use the YouTubeTranscriptQA tool to answer this question: {question}"
    full_answer = agent.run(prompt)
    short_answer = shorten_text(full_answer)
    return short_answer

# One interaction
def voice_rag_interaction():
    record_audio()
    question = transcribe_audio()
    answer = answer_with_sources(question)
    print(f"🤖 Answer:\n{answer}")
    speak_text(answer)


In [4]:
while True:
    voice_rag_interaction()
    cont = input("Ask another question? (y/n): ")
    if cont.strip().lower() != "y":
        break

🎙️ Recording...
🎙️ Recording finished
📝 Transcribing...


/opt/anaconda3/envs/cleanenv/lib/python3.10/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🗣️ You said: That is a black hole.


> Entering new AgentExecutor chain...


/var/folders/hq/l56ghxv518j9wg6pgqkbbvd80000gn/T/ipykernel_4572/3478499161.py:51: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  full_answer = agent.run(prompt)


Thought: The input doesn't seem to be a question about a YouTube transcript. It seems to be a statement about a black hole. However, I'll try to provide a response based on the given instruction.

Action: YouTubeTranscriptQA
Action Input: That is a black hole. 

/Users/test/Desktop/ironhack_labs/YouTube_ChatBot_Final/notebooks/utils.py:40: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": input_text})



Observation: A black hole is a region of spacetime where gravity is so strong that nothing—no particles or even electromagnetic radiation such as light—can escape from it. The theory of general relativity predicts that a sufficiently compact mass can deform spacetime to form a black hole.

Sources:
52 - The Sounds of Space ｜ StarTalk Live! at Guild Hall.en.txt
17 - When Black Holes Collide, with Nergis Mavalvala.en.txt
27 - The Most Energetic Events in the Universe with Tim Paglione.en.txt
40 - Neil deGrasse Tyson and Bill Nye Catch Up.en.txt
15 - Answering Questions about Higher Dimensions, Magnetic Poles & Mercury’s Core.en.txt
Thought:Do I need to use a tool? No
AI: A black hole is a region of spacetime where gravity is so strong that nothing—no particles or even electromagnetic radiation such as light—can escape from it. The theory of general relativity predicts that a sufficiently compact mass can deform spacetime to form a black hole. This information is sourced from various You